# imports

In [ ]:
from dataclasses import dataclass, field
import pygraphviz as pgv
import pandas as pd
INF = float('inf')

# решение

In [ ]:
@dataclass
class Graph:

    weights : dict[tuple[int, int], float]
    _graph: pgv.AGraph | None = field(default = None, init = False)
    _adjacency_matrix: pd.DataFrame | None = field(default = None, init = False)
    _path_matrix: pd.DataFrame | None = field(default = None, init = False)
    _vert_matrix: pd.DataFrame | None = field(default = None, init = False)

    @property
    def graph(self):
        if self._graph is None:
            self._generate_graph()
        return self._graph

    @property
    def adjacency_matrix(self):
        if self._adjacency_matrix is None:
            self._generate_tables()
        return self._adjacency_matrix

    @property
    def path_matrix(self):
        if self._path_matrix is None:
            self._generate_tables()
        return self._path_matrix

    @property
    def vert_matrix(self):
        if self._vert_matrix is None:
            self._generate_tables()
        return self._vert_matrix
    
    def _generate_graph(self):
        ...
    # (1) По алгоритму Флойда-Уоршалла найти матрицу весов маршрутов И матрицу номеров
    # первых вершин этих маршрутов.
    def _generate_tables(self):
        
        vs = set[int]()
        d = dict[str, list[float]]()
        p = dict[str, list[str]]()
        for k, v in self.weights.items():
            vs |= set(k)
        length = max(vs)

        for j in range(1, length+1):
            d[f'v{j}'] = [INF]*length
            p[f'v{j}'] = [f'v{j}']*length
            d[f'v{j}'][j-1] = 0

        for k, v in self.weights.items():
            v1, v2 = tuple(k)
            d[f'v{v1}'][v2-1] = v
            d[f'v{v2}'][v1-1] = v
        for j in range(1, length+1):
            for i in range(length):
                if d[f'v{j}'][i] == INF:
                    p[f'v{j}'][i] = '0'

        
        self._adjacency_matrix = pd.DataFrame(d , index=[f'v{i}' for i in range(1, length+1)])
        L = self._adjacency_matrix.copy()
        P = pd.DataFrame(p , index=[f'v{i}' for i in range(1, length+1)])
        for k in range(length):
            for i in range(length):
                vi, vk = f'v{i+1}', f'v{k+1}'
                if i != k and L.loc[vi, vk] != INF:
                    for j in range(length):
                        vj = f'v{j+1}'
                        if j != k and L.loc[vk, vj] != INF:
                            T: float = L.loc[vi, vk] + L.loc[vk, vj] # type: ignore
                            if T == INF or T < L.loc[vi, vj]: # type: ignore
                                L.loc[vi, vj] = T
                                P.loc[vi, vj] = P.loc[vi, vk]
        self._path_matrix = L
        self._vert_matrix = P

    # (2) Для указанной пары вершин vb ---> vt извлечь из матрицы маршрутов маршрут от
    # вершины vb к вершине vt в виде последовательности номеров вершин
    def get_path(self, vb: str, vt: str):
        path: list[str] = [vb]
        while path[-1] != vt:
            path.append(self.vert_matrix.loc[path[-1], vt]) # type: ignore
        return path


In [ ]:
var1 = {
     tuple({1,2}):11, tuple({1,7}):10, tuple({2,3}):31, tuple({2,6}):25, tuple({2,8}):21, 
tuple({3,5}):3, tuple({3,6}):16, tuple({4,5}):20, tuple({4,6}):14, tuple({5,8}):22, 
tuple({6,7}):19, tuple({6,8}):1, tuple({7,8}):5
}

In [ ]:
v1 = Graph(var1)
print(v1.adjacency_matrix)
print(v1.path_matrix)
print(v1.vert_matrix)
print(v1.weights)

In [ ]:
# v4 -> v7
print(v1.vert_matrix)
print(v1.vert_matrix.loc['v4','v7'])
print(v1.vert_matrix.loc['v6','v7'])
print(v1.vert_matrix.loc['v8','v7'])
print(v1.get_path('v4', 'v7'))